 # Segmenting and Clustering Neighborhoods in Toronto
 
## Question 1

Create a dataframe with **three columns**: PostalCode, Borough, and Neighborhood

-    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
-    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

-    If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
-    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
-    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [3]:
soup = BeautifulSoup(source, 'xml')

In [4]:
table=soup.find('table')

In [5]:
soup = BeautifulSoup(source, 'xml')

In [6]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for tr_cell in table.find_all('tr'):
    for td_cell in tr_cell.find_all('td'):
        tag = td_cell.findAll('a')
        if(td_cell.a != None):
            row = [td_cell.b.text]
            for i in tag:
                if(len(row)<=2):
                    row.append(i.text) 
                else:
                    row[2] = row[2] + ', ' + i.text
            if(len(row)==2):
                row.append(row[1])
            df.loc[len(df)] = row

In [7]:
df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
96,M8X,Etobicoke,"The Kingsway, Old Mill"
97,M4Y,Downtown Toronto,Church and Wellesley
98,M7Y,Business reply mail,Business reply mail
99,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu..."


In [8]:
df = df.groupby(["Postalcode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
print("Shape: ", df.shape)

Shape:  (101, 3)


## Question 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [10]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'Postalcode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:
- to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
- to generate maps to visualize your neighborhoods and how they cluster together. 

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [12]:
import numpy as np
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!pip install folium
import folium # map rendering library

!pip install geopy
from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 3.1 Get coordinates of Toronto City and create a map

In [13]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

## 3.2 Visualise neighborhoods in Toronto

In [15]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 3.3 Visualise neighborhoods whose Borough names contain Toronto

In [16]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4R,North Toronto,North Toronto,43.715383,-79.405678
1,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
2,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


In [17]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc) 
    
map_toronto_denc

## 3.4 Exploring neighborhoods whose Borough names contain Toronto

In [18]:
# Removed details for privacy
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

In [19]:
neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-21-9de588fd8d99>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Nailsense,Spa,43.717467,-79.400653
1,lululemon,Clothing Store,43.713478,-79.400082
2,A&W,Fast Food Restaurant,43.715149,-79.399944
3,Uncle Betty's Diner,Diner,43.714452,-79.400091
4,C'est Bon,Chinese Restaurant,43.716785,-79.400406
5,Tim Hortons,Coffee Shop,43.714894,-79.399776
6,Starbucks,Coffee Shop,43.715590,-79.400450
7,Second Cup,Café,43.714583,-79.400120
8,Sporting Life,Sporting Goods Shop,43.716277,-79.400248
9,St. Clements - Yonge Parkette,Park,43.712062,-79.404255


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighborhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )

In [24]:
toronto_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,62,62,62,62,62,62
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",16,16,16,16,16,16
Church and Wellesley,69,69,69,69,69,69
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,146,146,146,146,146,146
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",59,59,59,59,59,59


In [25]:
# one hot encoding
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = toronto_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Bay Street,0.016129,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0,0.0,0.0,0.016129,0.0,0.016129,0.0,0.016129,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2,Church and Wellesley,0.014493,0.014493,0.014493,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.014493,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.014493
3,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0,0.0,0.0,0.020000,0.0,0.000000,0.0,0.010000,0.000000
4,Downtown Toronto,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0,0.0,0.0,0.027397,0.0,0.000000,0.0,0.000000,0.000000


In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Café,Italian Restaurant,Restaurant,Bank,Pizza Place,Burger Joint
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Rental Car Location
2,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Gym,Fast Food Restaurant
3,"Commerce Court, Victoria Hotel",Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Deli / Bodega,Cocktail Bar
4,Downtown Toronto,Coffee Shop,Sandwich Place,Cocktail Bar,Café,Italian Restaurant,Bakery,Beer Bar,Park,Japanese Restaurant,Farmers Market


## 3.5 Performing k-means clustering on neighborhoods

In [29]:
# set number of clusters
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 4, 1, 4, 1, 1, 1, 4, 2])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4R,North Toronto,North Toronto,43.715383,-79.405678,2,Coffee Shop,Sporting Goods Shop,Ice Cream Shop,Fast Food Restaurant,Diner,Park,Pet Store,Clothing Store,Chinese Restaurant,Café
1,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Molecular Gastronomy Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant
2,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,4,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Café,Bakery,Pub,Convenience Store,Sandwich Place,Caribbean Restaurant
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Gym,Fast Food Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Spa,Mexican Restaurant,French Restaurant,Breakfast Spot


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Rental Car Location


In [33]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Spa,Mexican Restaurant,French Restaurant,Breakfast Spot
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Bank,Cosmetics Shop,Pizza Place,Japanese Restaurant,Theater
8,Downtown Toronto,1,Coffee Shop,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Café,Italian Restaurant,Restaurant,Bank,Pizza Place,Burger Joint
9,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Clothing Store,Gym,Restaurant,Sushi Restaurant,Japanese Restaurant,Burrito Place,Concert Hall
10,Downtown Toronto,1,Coffee Shop,Café,Hotel,Scenic Lookout,Pizza Place,Gym,Aquarium,Bank,Restaurant,Brewery
11,Downtown Toronto,1,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Japanese Restaurant,Salad Place,Pharmacy,Deli / Bodega,Restaurant
12,Downtown Toronto,1,Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Deli / Bodega,Cocktail Bar
13,Downtown Toronto,1,Café,Sandwich Place,Coffee Shop,Bar,Bakery,Japanese Restaurant,Pub,French Restaurant,Beer Store,Beer Bar
17,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Hotel,Asian Restaurant,Japanese Restaurant,Gym,Bank,Deli / Bodega,Sushi Restaurant


In [34]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Toronto,2,Coffee Shop,Sporting Goods Shop,Ice Cream Shop,Fast Food Restaurant,Diner,Park,Pet Store,Clothing Store,Chinese Restaurant,Café


In [35]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,3,Park,Playground,Trail,Molecular Gastronomy Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [36]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Café,Bakery,Pub,Convenience Store,Sandwich Place,Caribbean Restaurant
3,Downtown Toronto,4,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Gym,Fast Food Restaurant
6,Downtown Toronto,4,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Gastropub,Clothing Store,Restaurant,Cosmetics Shop,Beer Bar,Seafood Restaurant
7,Downtown Toronto,4,Coffee Shop,Sandwich Place,Cocktail Bar,Café,Italian Restaurant,Bakery,Beer Bar,Park,Japanese Restaurant,Farmers Market
14,Downtown Toronto,4,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Burger Joint,Mexican Restaurant,Gaming Cafe,Bar,Thai Restaurant,Poutine Place
16,Downtown Toronto,4,Coffee Shop,Sandwich Place,Cocktail Bar,Café,Italian Restaurant,Bakery,Beer Bar,Park,Japanese Restaurant,Farmers Market
18,Downtown Toronto,4,Coffee Shop,Sandwich Place,Cocktail Bar,Café,Italian Restaurant,Bakery,Beer Bar,Park,Japanese Restaurant,Farmers Market
